In [4]:
# ONLY RUN THIS ONCE. Uncomment right command for your OS before running.
# !source .venv/bin/activate (MacOS / Linux)
# !.venv\Scripts\activate (Windows)
!pip3 install torch torchvision
!pip install numpy
!pip install pyserial
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 11.5 MB/s eta 0:00:00a 0:00:01


# The Actual Code Functionality

In [1]:
# Check if your camera is working (ignore deprecation warning)
!python3 -c "import cv2; cap = cv2.VideoCapture(0); print(cap.isOpened())"
# If True, proceed. If not, try:
# 1. run !system_profiler SPCameraDataType to list existing cameras
# 2. Go to system preferences -> privacy & security -> camera -> check if your IDE has access to camera
# FYI: cv.VideoCapture(0) doesn't need a proxy like Photobooth. It takes images straight from your camera.

2025-01-30 08:17:35.787 Python[10459:132945] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
True


In [8]:
import cv2
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

# Load pre-trained model
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
model.eval()  # Set to evaluation mode

# Init preprocess method
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Use logging to classify errors and info messages
import logging
import time
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # Open camera and capture image
    cap = cv2.VideoCapture(0) # 0 is your computer's default camera
    logging.info(f"Camera is opened: {cap.isOpened()}")
    time.sleep(1)
    if not cap.isOpened():
         raise RuntimeError("Cannot open camera - check if camera is connected")


    ret, frame = cap.read() # ret: bool, frame: np.ndarray (matrix of pixels)
    if not ret or frame is None:
        raise RuntimeError("Failed to capture frame from camera")
    # Look for 'test_capture.jpg' in the notebooks folder
    cv2.imwrite("test_capture.jpg", frame)
    logging.info("Frame captured and saved as test_capture.jpg")

    # Pass frame through reprocess and then classify frame
    input_tensor = preprocess(frame).unsqueeze(0)  # Add batch dimension e.g., [height, width, channels] -> [batch_size, h, w, c] [1, ...] means process 1 image at a time.
    
except Exception as e:
    logging.error(f"Camera error: {e}")
    raise

finally:
    cap.release() # release camera resources after use

with torch.no_grad():
        output = model(input_tensor)
        class_id = torch.argmax(output).item()  # Get predicted class ID


### Draft Code for Serial Communication with Arduino

```python
import serial

# Initialize serial communication
arduino = serial.Serial('COM3', 9600)  # Adjust port and baud rate

# Capture image
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

# Preprocess and classify
input_tensor = preprocess(frame).unsqueeze(0)  # Add batch dimension
with torch.no_grad():
    output = model(input_tensor)
class_id = torch.argmax(output).item()  # Get predicted class ID

# Send signal to Arduino
arduino.write(str(class_id).encode())

cap.release()
```